In [1]:
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
import pyarrow.parquet as pq
import datetime as dt

from sympy import symbols, Symbol, Eq, nsolve, solve, sqrt
from decimal import Decimal, getcontext

import multiprocessing as mp

from LP_Action_Liquidity_Parameter_Function import Get_Liquidity_Parameter, Get_Price

data_dir = '/Users/zhicong/Dropbox/DEXs'
pool_name = 'WETH-USDT 100 1'

# LP Action

We import all LP actions and conduct basic cleaning.

In [2]:
lp_action = pd.read_csv(os.path.join(data_dir, "uniswapv3_lp_actions", pool_name + ".csv"))
lp_action.columns = [x.upper() for x in lp_action.columns.tolist()]
lp_action.dropna(subset = ['AMOUNT0_USD','AMOUNT1_USD'], inplace = True)

# Final cleaning
lp_action['BLOCK_NUMBER'] = lp_action['BLOCK_NUMBER'].astype(int)
lp_action.sort_values(by = ['BLOCK_NUMBER'], ascending = False, inplace = True)
lp_action['AMOUNT0_ADJUSTED'] = lp_action['AMOUNT0_ADJUSTED'].astype(float)
lp_action['AMOUNT1_ADJUSTED'] = lp_action['AMOUNT1_ADJUSTED'].astype(float)
lp_action['BLOCK_TIMESTAMP'] = pd.to_datetime([x[:-5] for x in lp_action['BLOCK_TIMESTAMP'].astype(str)])
lp_action = lp_action[(lp_action['AMOUNT0_ADJUSTED'] != 0) | (lp_action['AMOUNT1_ADJUSTED'] != 0)]
lp_action['NF_TOKEN_ID'] = lp_action['NF_TOKEN_ID'].apply(str).str.replace('.0','', regex = False)

# Changing signs of addition and extraction
lp_action['ACTION'] = lp_action['ACTION'].replace({'INCREASE_LIQUIDITY':'Increase Liquidity',
                                                   'DECREASE_LIQUIDITY':'Decrease Liquidity'})
lp_action.rename(columns = {'LIQUIDITY_PROVIDER':'SENDER', 'ACTION':'EVENT_NAME'}, inplace = True)

# Get lower and upper price from tick
lp_action['PRICE_LOWER'] = lp_action['TICK_UPPER'].apply(Get_Price)
lp_action['PRICE_UPPER'] = lp_action['TICK_LOWER'].apply(Get_Price)

# Reverse to make sure LPs are sorted in the same order as they are executed
lp_action = lp_action.iloc[::-1]
lp_action.reset_index(drop = True, inplace = True)

print("Number of LP Actions in this pool:",len(lp_action))
lp_action.head()

Number of LP Actions in this pool: 9


,POOL_NAME,TX_HASH,EVENT_NAME,BLOCK_NUMBER,BLOCK_TIMESTAMP,SENDER,NF_TOKEN_ID,AMOUNT0_ADJUSTED,TOKEN0_SYMBOL,AMOUNT0_USD,AMOUNT1_ADJUSTED,TOKEN1_SYMBOL,AMOUNT1_USD,TICK_LOWER,TICK_UPPER,PRICE_LOWER,PRICE_UPPER
0,WETH-USDT 100 1,0x151e7360d01ccc21204b281064ce4fd3b9f808cff627...,Increase Liquidity,16266586,2022-12-26 04:37:47,0x611350945b214a8fba2c0270ac04a83c342e79b7,398381,27.257696,WETH,33324.714445,33252.433782,USDT,33252.433782,-214418,-196092,327900119205463785472,2049225966470396641280
1,WETH-USDT 100 1,0x64a091d54a078ff228888153301cafc80e14a834f2db...,Decrease Liquidity,16269858,2022-12-26 15:35:35,0x611350945b214a8fba2c0270ac04a83c342e79b7,398381,27.348674,WETH,33236.023364,33141.583286,USDT,33133.264749,-214418,-196092,327900119205463785472,2049225966470396641280
2,WETH-USDT 100 1,0x2169ae9a2d3b57bc55b3ab722efbd619aefbe3b8ff0b...,Increase Liquidity,16683792,2023-02-22 11:49:35,0xd6de95b50dab7153a3ea8f48486ffd677231eeb5,449289,0.000100,WETH,0.164135,0.000000,USDT,0.000000,-205279,-205270,820950783345634836480,821689934661897945088
3,WETH-USDT 100 1,0x36b3051efdcb3c689c87ffe09158c9f77152093678f3...,Increase Liquidity,16838920,2023-03-16 07:25:59,0x8d890268e48d96482f6f1adfb99ac138486e0b7c,466591,0.000880,WETH,1.463315,1.100000,USDT,1.100000,-205286,-205266,820622485110958718976,822265290199809523712
4,WETH-USDT 100 1,0xfcbddd5399791dc09e819e422b50bf5ad019ed2748b0...,Increase Liquidity,16919349,2023-03-27 14:35:35,0xf25fbc15a3ce8fbe040517748a92e5030ef6839e,477147,0.088644,WETH,154.862321,50.000000,USDT,49.892850,-209475,-194715,285721034778838237184,1250054134368222838784


# Match Withdrawal with Deposition

In [3]:
"""
token_transfer = pd.read_csv(os.path.join(data_dir, "uniswapv3_NFT", pool_name + ".csv"))
token_transfer.columns = [x.upper() for x in token_transfer.columns.tolist()]
token_transfer = token_transfer.sort_values(by = ['TOKENID','BLOCK_NUMBER'])

token_transfer = token_transfer[token_transfer['BLOCK_NUMBER'] <= lp_action['BLOCK_NUMBER'].max()]

token_transfer.head()

initial = token_transfer.loc[token_transfer['NFT_FROM_ADDRESS'] == '0x0000000000000000000000000000000000000000',['TOKENID','TX_HASH','NFT_TO_ADDRESS']]
initial['EVENT_NAME'] = 'Increase Liquidity'

final = token_transfer.groupby(['TOKENID'])[['TOKENID','TX_HASH','NFT_TO_ADDRESS']].tail(1)
final['EVENT_NAME'] = 'Decrease Liquidity'

token_transfer = pd.concat([initial,final], axis = 0)
token_transfer.sort_values(by = ['TOKENID'], inplace = True)
token_transfer.rename(columns = {'NFT_TO_ADDRESS':'SENDER'}, inplace = True)

token_transfer.head()
"""

'\ntoken_transfer = pd.read_csv(os.path.join(data_dir, "uniswapv3_NFT", pool_name + ".csv"))\ntoken_transfer.columns = [x.upper() for x in token_transfer.columns.tolist()]\ntoken_transfer = token_transfer.sort_values(by = [\'TOKENID\',\'BLOCK_NUMBER\'])\n\ntoken_transfer = token_transfer[token_transfer[\'BLOCK_NUMBER\'] <= lp_action[\'BLOCK_NUMBER\'].max()]\n\ntoken_transfer.head()\n\ninitial = token_transfer.loc[token_transfer[\'NFT_FROM_ADDRESS\'] == \'0x0000000000000000000000000000000000000000\',[\'TOKENID\',\'TX_HASH\',\'NFT_TO_ADDRESS\']]\ninitial[\'EVENT_NAME\'] = \'Increase Liquidity\'\n\nfinal = token_transfer.groupby([\'TOKENID\'])[[\'TOKENID\',\'TX_HASH\',\'NFT_TO_ADDRESS\']].tail(1)\nfinal[\'EVENT_NAME\'] = \'Decrease Liquidity\'\n\ntoken_transfer = pd.concat([initial,final], axis = 0)\ntoken_transfer.sort_values(by = [\'TOKENID\'], inplace = True)\ntoken_transfer.rename(columns = {\'NFT_TO_ADDRESS\':\'SENDER\'}, inplace = True)\n\ntoken_transfer.head()\n'

Merge token_transfer into lp_action.

# Add Liquidity Parameter

In [4]:
with mp.Pool(10) as pool:
    result = pool.imap(Get_Liquidity_Parameter, lp_action[['AMOUNT1_ADJUSTED','AMOUNT0_ADJUSTED','PRICE_LOWER','PRICE_UPPER']].iterrows(), chunksize = 1)
    output = [x for x in result]

lp_action['L'] = output

Working on row: 0


In [5]:
lp_action.to_csv(os.path.join(data_dir, "uniswapv3_lp_action_LP", pool_name + "_LP.csv"), index=False)